In [ ]:
from pymongo import MongoClient

CONNECTION_STRING = "mongodb://localhost:27017"
client = MongoClient(CONNECTION_STRING)
db = client["florence"]
collection = db["newProducts"]

In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint


products = collection.find({"reviews": {"$exists": True, "$ne": []}})

df = pd.DataFrame(list(products))
df=df[["id","reviews","starRatings"]]

csv_file = "reviews.csv"
df.to_csv(csv_file, index=False, encoding='utf-8')

In [ ]:
from ast import literal_eval
Df = pd.read_csv("reviews.csv")
Df['reviews'] = Df['reviews'].apply(literal_eval)
Df

In [44]:
from huggingface_hub import InferenceApi
from transformers import pipeline, AutoTokenizer

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = pipeline("sentiment-analysis", model=model_path, tokenizer=tokenizer)

# model = InferenceApi("cardiffnlp/twitter-roberta-base-sentiment-latest")


def get_polarity(text):
    result ={}
    result["positive"] = 0
    result["negative"] = 0
    result["neutral"] = 0
    output = model(text)
    for outcome in output:
        if outcome["label"] == "negative":
            result["negative"] += outcome["score"]
        if outcome["label"] == "positive":
            result["positive"] += outcome["score"]
        if outcome["label"] == "neutral":
            result["neutral"] += outcome["score"]
    return result


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
import csv  

output_csv_file = "output_results.csv"

# Open the CSV file in write mode
file = open(output_csv_file, 'w', newline='', encoding='utf-8')

# Create a CSV writer object
csv_writer = csv.writer(file)

try:
    # Write the header row
    csv_writer.writerow(["pid", "polarity_positive","polarity_negative","polarity_neutral", "star_ratings"])

    # Iterate through the DataFrame
    for ind in Df.index:
        pid = Df.at[ind, "id"]
        reviewList = Df.at[ind, "reviews"]
        
        # Average polarity of reviews
        result = {}
        result["positive"] = 0
        result["negative"] = 0
        result["neutral"] = 0

        try:
            for review in reviewList:
                output = get_polarity(review["review"])
                result["positive"] += output["positive"]
                result["neutral"] += output["neutral"]
                result["negative"] += output["negative"]

            result["positive"] /= len(reviewList)
            result["negative"] /= len(reviewList)
            result["neutral"] /= len(reviewList)

            # Get the starRatings
            ratings = Df.at[ind, "starRatings"]

            # Write the row to the CSV file
            csv_writer.writerow([pid, result["positive"],result["negative"],result["neutral"], ratings])

        except Exception as e:
            print(f"Error processing row {ind}: {e}")

finally:
    # Close the file in the finally block to ensure it's closed even if an exception occurs
    file.close()